# BTC Strategy Design
***
## Import Package and Data

In [1]:
from datetime import datetime
from reference.Strategy import BaseStrategyFrame
from reference.Strategy import zwpy_sta
import pathlib
import backtrader as bt
import pyfolio as pf
import pandas as pd

# route init
curr_folder = pathlib.Path().cwd()
BTC_data_min = curr_folder / "BTCUSDT_UPERP_1m.csv"

# report folder init
report_folder = curr_folder / "report"
if not report_folder.exists():
    report_folder.mkdir(parents=True, exist_ok=True)

data_folder = curr_folder / "data"
if not data_folder.exists():
    data_folder.mkdir(parents=True, exist_ok=True)

cerebro = bt.Cerebro()
cerebro.addstrategy(zwpy_sta.TurStrategy)
cerebro.broker.setcash(100000)

/home/thomas/Desktop/BTC/env/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


## Data Resample

In [2]:
BTC_data_min = pd.read_csv(BTC_data_min, parse_dates =["datetime"], index_col ="datetime")
print("Minute ohlc\n", BTC_data_min.head(10))
BTC_data = BTC_data_min.resample('h').mean()
print("\nHour ohlc\n", BTC_data.head(10))
BTC_data.to_csv(str(data_folder / "BTC_hour.csv"))
BTC_data = pathlib.Path().cwd() / "data" / "BTC_hour.csv"


Minute ohlc
                         open     high      low    close  volume
datetime                                                       
2019-09-25 15:44:00  8323.22  8348.51  8323.22  8345.08  15.878
2019-09-25 15:45:00  8346.57  8348.32  8339.09  8339.09  15.931
2019-09-25 15:46:00  8340.21  8341.79  8320.05  8320.77  29.408
2019-09-25 15:47:00  8320.77  8338.87  8320.77  8336.79  40.464
2019-09-25 15:48:00  8336.79  8342.28  8329.25  8340.73  34.654
2019-09-25 15:49:00  8343.46  8343.89  8338.15  8343.86  33.772
2019-09-25 15:50:00  8341.07  8364.00  8341.07  8360.93  43.837
2019-09-25 15:51:00  8359.17  8379.33  8352.64  8374.18  43.769
2019-09-25 15:52:00  8375.08  8380.26  8367.21  8374.61  55.568
2019-09-25 15:53:00  8374.02  8374.02  8343.37  8346.06  26.116

Hour ohlc
                             open         high          low        close  \
datetime                                                                  
2019-09-25 15:00:00  8343.848750  8353.017500  8334.82062

## Data format & Feed

In [3]:
dt_start = datetime.strptime("2019-09-25","%Y-%m-%d")
dt_end = datetime.strptime("2021-10-28","%Y-%m-%d")
data = bt.feeds.GenericCSVData(
    timeframe = bt.TimeFrame.Minutes,
    compression = 60,
    dataname=BTC_data,
    fromdate=dt_start,      
    todate=dt_end,
    nullvalue=0.0,
    dtformat=('%Y-%m-%d %H:%M:%S'),   
    datetime=0,          
    open = 1,
    high = 2,
    low = 3,
    close = 4,
    openinterest=-1,
    volume = -1
)
cerebro.adddata(data)

## Run Backtrader

In [4]:
print('Starting Value: %.2f' % cerebro.broker.getvalue())
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
results = cerebro.run()
print('Ending Value: %.2f' % cerebro.broker.getvalue())
strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
cerebro.plot()

Starting Value: 100000.00
printlog: False
n_high: 30
n_low: 15
=== Backtesting Start! ===
=== Backtesting Finished! ===
Ending Value: 123160.64


<IPython.core.display.Javascript object>

[[<Figure size 432x288 with 6 Axes>]]

## Pyfolio

In [5]:
pf.create_full_tear_sheet(returns)

Start date,2019-09-25
End date,2021-10-28
Total months,36
,Backtest
Annual return,7.103%
Cumulative returns,23.161%
Annual volatility,10.214%
Sharpe ratio,0.72
Calmar ratio,0.38
Stability,0.61
Max drawdown,-18.667%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,18.67,2021-03-09,2021-06-08,NaT,NaN
1,4.09,2021-01-14,2021-01-26,2021-01-29,12
2,3.63,2021-01-29,2021-02-01,2021-02-06,6
3,2.80,2021-02-09,2021-02-16,2021-02-20,9
4,2.18,2021-02-21,2021-02-26,2021-03-09,12


<IPython.core.display.Javascript object>

/home/thomas/Desktop/BTC/env/lib/python3.8/site-packages/pyfolio/timeseries.py:1230: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  period = returns_dupe.loc[start:end]


Stress Events,mean,min,max
New Normal,0.03%,-3.53%,4.38%


<IPython.core.display.Javascript object>

## Strategy

    Rule:
        If close price > max( high price of pass n days): buy.
        After buy action, if close price < min( low proce of pass n day): sell

    Args:
        n_high(int): highest high price of pass n day.
        n_low(int): lowest low price of pass n day.


In [ ]:
class TurStrategy(BaseStrategyFrame):

    params = (("n_high", 30), ("n_low", 15))

    def __init__(self):

        # multiple inheritance
        super(TurStrategy, self).__init__()

        print("printlog:", self.params.printlog)
        print("n_high:", self.params.n_high)
        print("n_low:", self.params.n_low)

        # Add indicators
        self.pass_highest = bt.indicators.Highest(
            self.datahigh, period=self.params.n_high
        )

        self.pass_lowest = bt.indicators.Lowest(self.datalow, period=self.params.n_low)

    def next(self):
        # Simply log the closing price of the series from the reference
        # self.log("Close, %.2f" % self.dataclose[0])
        self.log(
            "O:{:.2f}, H:{:.2f}, L:{:.2f}, C:{:.2f}".format(
                self.dataopen[0], self.datahigh[0], self.datalow[0], self.dataclose[0]
            )
        )

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.pass_highest[-1]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log("BUY CREATE, %.2f" % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.pass_lowest[-1]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log("SELL CREATE, %.2f" % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
